In [5]:
# pip install pandas scikit-learn numpy
import pandas as pd


In [65]:
ratings = pd.read_csv(r'ml-latest-small\ml-latest-small\ratings.csv')
movies = pd.read_csv(r'ml-latest-small\ml-latest-small\movies.csv')

# ratings.head()
# movies.head()
ratings_movies = pd.merge(ratings, movies, on='movieId')
ratings_movies.head()


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [66]:
# Create a pivot table (user-item matrix)
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')

# Fill NaN values with 0 (indicating unrated movies)
user_item_matrix = user_item_matrix.fillna(0)

print(user_item_matrix.head())

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity matrix
similarity_matrix = cosine_similarity(user_item_matrix)

# Convert the similarity matrix to a DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# Find similar users to a given user (for example, userId = 1)
user_similarities = similarity_df[1].sort_values(ascending=False)

# Get the top 5 similar users
top_users = user_similarities.head(6)
print(top_users)


userId
1      1.000000
266    0.357408
313    0.351562
368    0.345127
57     0.345034
91     0.334727
Name: 1, dtype: float64


In [68]:
# Create a movie-item matrix (movies × users)
movie_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')

# Fill NaN values with 0
movie_item_matrix = movie_item_matrix.fillna(0)

# Calculate movie similarity matrix
movie_similarity = cosine_similarity(movie_item_matrix)

# Convert to DataFrame
movie_similarity_df = pd.DataFrame(movie_similarity, index=movie_item_matrix.index, columns=movie_item_matrix.index)


In [69]:
# Find similar movies to a given movieId (for example, movieId = 1)
movie_similarities = movie_similarity_df[1].sort_values(ascending=False)

# Get the top 5 similar movies
top_movies = movie_similarities.head(6)
print(top_movies)


movieId
1       1.000000
3114    0.572601
480     0.565637
780     0.564262
260     0.557388
356     0.547096
Name: 1, dtype: float64


In [70]:
# One-hot encode the genres
genre_matrix = movies['genres'].str.get_dummies('|')

# Merge the genre matrix with ratings
movies_with_genres = pd.concat([movies, genre_matrix], axis=1)

print(movies_with_genres.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  (no genres listed)  Action  \
0  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
1                   Adventure|Children|Fantasy                   0       0   
2                               Comedy|Romance                   0       0   
3                         Comedy|Drama|Romance                   0       0   
4                                       Comedy                   0       0   

   Adventure  Animation  Children  Comedy  Crime  ...  Film-Noir  Horror  \
0          1          1         1       1      0  ...          0       0   
1          1          0         1       0      0  ...          0       0   
2 

In [72]:
# Calculate movie genre similarity
genre_similarity = cosine_similarity(genre_matrix)

# Convert to DataFrame
genre_similarity_df = pd.DataFrame(genre_similarity, index=movies['movieId'], columns=movies['movieId'])


In [73]:
# Get similar movies for a liked movie (e.g., movieId = 1)
similar_movies_genre = genre_similarity_df[1].sort_values(ascending=False)

# Top 5 similar movies
print(similar_movies_genre.head())


movieId
1         1.0
103755    1.0
4886      1.0
91355     1.0
136016    1.0
Name: 1, dtype: float64
